In [2]:
import pandas as pd

# 국세청 승인자료 면세파트 만들기

In [3]:
data = pd.read_excel('가상계좌 확인용.xlsx')

In [4]:
# 조건에 따라 실공급가액 및 면세 컬럼 계산 및 추가
data['실공급가액'] = data.apply(lambda x: x['부가세']*10 if x['부가세'] != 0 else 0, axis=1)
data['봉사료'] = data.apply(lambda x: x['공급가액'] - x['실공급가액'] if x['부가세'] != 0 else x['공급가액'], axis=1)

In [5]:
# 면세가 음수인 경우, 해당 금액을 실공급가액에 더하고 면세를 0으로 업데이트
data['실공급가액'] = data.apply(lambda x: x['실공급가액'] + x['봉사료'] if x['봉사료'] < 0 else x['실공급가액'], axis=1)
data['봉사료'] = data.apply(lambda x: 0 if x['봉사료'] < 0 else x['봉사료'], axis=1)

In [6]:
data['실공급가액'] += data['봉사료'] % 10
data['봉사료'] -= data['봉사료'] % 10
data['공급가액'] = data['실공급가액']
data.drop('실공급가액', axis=1, inplace=True)
data.to_excel('가상계좌 확인용_면세금액정리.xlsx', index=False)

In [7]:
# 파일 경로
file_path_tax_approval = '가상계좌 확인용_면세금액정리.xlsx'
file_path_credit_card = '신용·체크카드_결제내역_2024.05.01-2024.06.01.xlsx'
file_path_cash_receipt = '현금영수증_2024.05.01-2024.06.01.xlsx'

# 각 파일 읽기
df_tax_approval = pd.read_excel(file_path_tax_approval)
df_credit_card = pd.read_excel(file_path_credit_card)
df_cash_receipt = pd.read_excel(file_path_cash_receipt)

C:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
df_credit_card.info()
df_tax_approval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상점아이디(MID)  91 non-null     object
 1   결제일시        91 non-null     object
 2   취소일시        8 non-null      object
 3   주문번호        91 non-null     object
 4   결제상태        91 non-null     object
 5   구매자명        91 non-null     object
 6   결제액         91 non-null     int64 
 7   취소액         91 non-null     int64 
 8   부가세         91 non-null     int64 
 9   공급가액        91 non-null     int64 
 10  면세가액        91 non-null     int64 
 11  결제수단        91 non-null     object
 12  결제기관        91 non-null     object
 13  구매상품        91 non-null     object
 14  안내메시지       91 non-null     object
 15  승인번호        91 non-null     int64 
 16  취소자         8 non-null      object
dtypes: int64(6), object(11)
memory usage: 12.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data column

In [9]:
# 국세청 데이터와 신용카드 결제 내역 데이터를 합치는 과정
# df_tax_approval은 국세청 데이터, merged_data는 신용카드 결제 내역 데이터를 나타냅니다.

# 새로운 데이터프레임 생성을 위해 필요한 컬럼만 df_2에서 선택
merged_data = df_credit_card[['결제일시', '공급가액', '부가세','면세가액', '결제액', '승인번호', '결제수단','주문번호']].copy()

# 컬럼 이름을 df_1과 일치시키기 위해 변경
merged_data.rename(columns={
    '결제일시': '매출일시',
    '결제액': '총금액',
    '결제수단': '거래구분',
    '면세가액': '봉사료'
}, inplace=True)

# 봉사료, 신분확인 뒷4자리, 용도구분, 비고 컬럼 추가 (값은 비워둠)
merged_data['신분확인뒷4자리'] = None  # 신분확인 뒷4자리는 비워둡니다.
merged_data['용도구분'] = None  # 용도구분은 비워둡니다.
merged_data['비고'] = None  # 비고는 비워둡니다.

# df_1 (국세청 데이터)과 합치기
final_df = pd.concat([df_tax_approval, merged_data], ignore_index=True)

# 최종 데이터프레임 확인
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1346 entries, 0 to 1345
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1255 non-null   object 
 1   결제 취소 일시    1255 non-null   object 
 2   봉사료         1346 non-null   int64  
 3   공급가액        1346 non-null   int64  
 4   부가세         1346 non-null   int64  
 5   총금액         1346 non-null   int64  
 6   승인번호        1346 non-null   int64  
 7   Unnamed: 7  1255 non-null   float64
 8   Unnamed: 8  1255 non-null   object 
 9   Unnamed: 9  1255 non-null   object 
 10  주문번호        91 non-null     object 
 11  이름          0 non-null      float64
 12  아이디         0 non-null      float64
 13  매출일시        91 non-null     object 
 14  거래구분        91 non-null     object 
 15  신분확인뒷4자리    0 non-null      object 
 16  용도구분        0 non-null      object 
 17  비고          0 non-null      object 
dtypes: float64(3), int64(5), object(10)
memory usage: 189.4+ KB


In [10]:
# 승인번호를 기준으로 신용체크카드 결제내역과 현금영수증 내역의 주문번호를 추출
cash_receipt_order_numbers = df_cash_receipt[['승인번호', '주문번호']].drop_duplicates(subset='승인번호')

# 국세청 승인자료에 주문번호 컬럼 추가하기
Ordernumber_merged = final_df.merge(cash_receipt_order_numbers, on='승인번호', how='left')

In [11]:
# '주문번호_x' 컬럼의 NaN 값을 '주문번호_y' 값으로 채우고, '주문번호'라는 새 컬럼에 저장
Ordernumber_merged['주문번호'] = Ordernumber_merged['주문번호_x'].fillna(Ordernumber_merged['주문번호_y'])

# 이제 '주문번호_x'와 '주문번호_y' 컬럼은 필요 없으므로 삭제
Ordernumber_merged.drop(columns=['주문번호_x', '주문번호_y'], inplace=True)

In [12]:
Ordernumber_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1346 entries, 0 to 1345
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1255 non-null   object 
 1   결제 취소 일시    1255 non-null   object 
 2   봉사료         1346 non-null   int64  
 3   공급가액        1346 non-null   int64  
 4   부가세         1346 non-null   int64  
 5   총금액         1346 non-null   int64  
 6   승인번호        1346 non-null   int64  
 7   Unnamed: 7  1255 non-null   float64
 8   Unnamed: 8  1255 non-null   object 
 9   Unnamed: 9  1255 non-null   object 
 10  이름          0 non-null      float64
 11  아이디         0 non-null      float64
 12  매출일시        91 non-null     object 
 13  거래구분        91 non-null     object 
 14  신분확인뒷4자리    0 non-null      object 
 15  용도구분        0 non-null      object 
 16  비고          0 non-null      object 
 17  주문번호        1346 non-null   object 
dtypes: float64(3), int64(5), object(10)
memory usage: 199.8+ KB


In [13]:
# 주문번호에서 '-'와 '-' 뒤의 값을 삭제
Ordernumber_merged['주문번호'] = Ordernumber_merged['주문번호'].str.split('-').str[0]

In [14]:
# 결과를 새 파일로 저장
output_file_path = '국세청승인자료_주문번호추가.xlsx'
Ordernumber_merged.to_excel(output_file_path, index=False)

# 저장된 파일 경로 반환
output_file_path

'국세청승인자료_주문번호추가.xlsx'

In [15]:
Ordernumber_merged

,Unnamed: 0,결제 취소 일시,봉사료,공급가액,부가세,총금액,승인번호,Unnamed: 7,Unnamed: 8,Unnamed: 9,이름,아이디,매출일시,거래구분,신분확인뒷4자리,용도구분,비고,주문번호
0,사업자,2024-05-01 11:15:49,1400,0,0,1400,160386640,1234.0,승인거래,소비자소득공제용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000002927
1,사업자,2024-05-07 17:46:05,0,2182,218,2400,160688069,1234.0,승인거래,소비자소득공제용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000000876
2,사업자,2024-05-01 16:02:42,0,3091,309,3400,160652547,1234.0,승인거래,소비자소득공제용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000000834
3,사업자,2024-05-17 12:39:41,3500,0,0,3500,160464472,1234.0,승인거래,소비자소득공제용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000003633
4,사업자,2024-05-21 15:05:26,0,3636,364,4000,160558137,1234.0,승인거래,소비자소득공제용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000003799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,NaN,NaN,0,47273,4727,52000,179811,NaN,NaN,NaN,NaN,NaN,2024-05-02 10:52:20,일반,None,None,None,3000000837
1342,NaN,NaN,0,46091,4609,50700,64944567,NaN,NaN,NaN,NaN,NaN,2024-05-01 17:36:23,일반,None,None,None,3000000836
1343,NaN,NaN,276000,7273,727,284000,47493679,NaN,NaN,NaN,NaN,NaN,2024-05-01 16:29:52,일반,None,None,None,3000000835
1344,NaN,NaN,0,107636,10764,118400,64543614,NaN,NaN,NaN,NaN,NaN,2024-05-01 15:49:19,일반,None,None,None,3000000833


# 주문번호 붙이기 완성 이제 고객명 붙이기

In [17]:
# 데이터 파일 로드
dalat_sales = pd.read_excel('5월 정산_0501_final.xlsx')
tax_approval_data = pd.read_excel('국세청승인자료_주문번호추가.xlsx')

# 'Order No'와 '주문번호'를 기준으로 두 데이터 프레임을 합친다
merged_data = pd.merge(tax_approval_data, dalat_sales[['Order No', '고객 이름','operator']], left_on='주문번호', right_on='Order No', how='left')

# 중복된 행 제거
final_data = merged_data.drop_duplicates()

# 필요한 컬럼만 선택하여 최종 데이터 프레임 생성
final_columns = ['발행구분', '매출일시', '공급가액', '부가세', '봉사료', '총금액', '승인번호', '신분확인뒷4자리', '거래구분', '용도구분', '비고', '주문번호', '고객 이름','operator']
final_data = final_data[final_columns]

# 최종 데이터를 Excel 파일로 저장
final_data_path = 'final_merged_data.xlsx'
final_data.to_excel(final_data_path, index=False)

이후에 나온 엑셀 데이터는 현금영수증 + 신용카드임. 
1. 신용카드에도 마이너스 금액을 총금액으로 옮겨주어야 함
2. 여신 금액 가져와서 더 붙여주기

# 노가다해서 나온 결과물 groupby 해주기

In [163]:
import openpyxl

In [170]:
# 엑셀 파일 경로
file_path = '달랏 3월 원장.xlsx'

# 첫 번째 시트의 데이터를 읽어옴
df = pd.read_excel(file_path, sheet_name=0)

# '고객 이름'과 '비고'를 기준으로 그룹화하고, 해당 그룹별로 '공급가액', '부가세', '봉사료', '총금액'의 합계 계산
grouped_by_customer_remark = df.groupby(['고객 이름', '비고'])[['공급가액', '부가세', '봉사료', '총금액']].sum().reset_index()

In [171]:
# 최종 데이터를 Excel 파일로 저장
grouped_path = '고객별 데이터.xlsx'
grouped_by_customer_remark.to_excel(grouped_path, index=False)